In [1]:
#to get the files from google drive folder
!pip install kora -q
from kora import drive
drive.link_nbs()

     |████████████████████████████████| 57 kB 1.7 MB/s 
     |████████████████████████████████| 56 kB 2.5 MB/s 
Mounted at /content/drive


In [2]:
import pandas as pd
import requests
import io
import numpy as np  
from datetime import date, timedelta
import re
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
#from github import Github
#import github
import torch
import torch.nn as nn
# Import tensor dataset & data loader
from torch.utils.data import TensorDataset, DataLoader
# Import nn.functional
import torch.nn.functional as F
import torch.optim as optim
from typing import Union, Tuple
import os
import sys
import time
from collections import OrderedDict
from sklearn.preprocessing import MinMaxScaler
from statistics import mean
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score
import math
import random
import imageio
from data_preparation import create_dataset, data_preparation
from model_fit_code import fit
#from sklearn.metrics import mean_absolute_percentage_error
matplotlib.style.use('seaborn')
%matplotlib inline


In [4]:
#generalised implementation of LSTM 
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim,  output_dim,num_layers, seq_length):
        super(LSTM, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim # hidden fimension
        self.seq_length=seq_length # sequence length
        # Number of hidden layers
        self.num_layers = num_layers

        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True) # nn lstm layer
        self.relu = nn.ELU() # elu activation function 
        # Readout layer
        print(output_dim)
        self.fc = nn.Linear(hidden_dim*self.seq_length, output_dim) # fully connected layer

    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()

        # Initialize cell state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()

        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        x = out.contiguous().view(batch_size,-1)
  
        out = self.fc(self.relu(x)) 
        # out.size() --> 100, 10
        return out

In [5]:
Shortlisted_States=['Karnataka','Maharashtra','Uttar-Pradesh','Kerala','Tamil-Nadu'] #list of states 
results_lstm=[]
lstm_models=[]
for state in Shortlisted_States: # state iteration 
  best_models=[]
  # getting data from github
  df=pd.read_csv("https://raw.githubusercontent.com/sureshkuc/Data-Science-in-Life-Science-Project/main/Indian-States-Covid19-Datasets/"+state+".csv", parse_dates=["Date"]).drop(columns =["Unnamed: 0"])
  df = df[df["Date"] > "2020-03-10"] # selecting data from 10th March 2020 onwards
  df = df.set_index("Date")
  df = df[['Confirmed', 'Recovered', 'Deceased', 'New_Confirmerd', 'New_Deaths', 'New_Recovered']] # selected feature
  #print(df.describe())

  time_step=[5,7,15,30] # list of time step
  Number_of_feature=[1,2,3,4,5,6] # list of feature index
  multi_feature=True
  output_dim=1
  min_error=np.iinfo(0).max # min error initialization 
  lstm_best_model={}
  for n_f in Number_of_feature: # feature iteration 
    for t_s in time_step: # time step iteration 
      # data preprocessing
      train_loader, test_loader, scaler = data_preparation(df, scaling_range=(0,1),time_step=t_s,number_feature=n_f, response_variable_index=0,data_split_ratio=0.8, Suffle=False)
      for n_layers in range(1,2,1): #iteration  of layers
        for n_hidden_nodes in [1,5,8,16,32]: # iteration over list of hidden nodes
          # setting seed to reproduce the results
          #random.seed(42)
          #torch.manual_seed(42)
          #np.random.seed(42)
          max_epochs=100
          
          #CNN model with L1 loss
          #best_model=Call_CNN_model(state,dataset=(train_loader, test_loader), lr=1e-2,criterion=nn.L1Loss(),max_epochs=max_epochs)
          lstm_model = LSTM(n_f, n_hidden_nodes, output_dim, n_layers,t_s)
          #if torch.cuda.is_available():
          #stm_model = lstm_model.cuda()
          #print(lstm_model)
          lstm_optim = optim.SGD(lstm_model.parameters(), lr=1e-3, momentum=0.9)
          #fc_optim = optim.Adam(fc_model.parameters(), lr=1e-3)
          train_losses,test_losses,best_model = fit(lstm_model, lstm_optim,nn.L1Loss(),(train_loader, test_loader), max_epochs=max_epochs,cuda=False)
          #print(f'\nTraining took {end-start}s!')
          #plot_loss(max_epochs,train_losses,test_losses,model_name='CNN for '+state)
          lstm_model = LSTM(n_f, n_hidden_nodes, output_dim, n_layers,t_s)
          lstm_model.load_state_dict(best_model)
          lstm_model.eval()
          test_x,test_y=test_loader
          predictions=lstm_model(test_x)
          test_y=test_y.cpu().detach().numpy()
          predictions=predictions.cpu().detach().numpy()
          mae=mean_absolute_error(test_y,predictions)
          rmse=math.sqrt(mean_squared_error(test_y,predictions))
          if rmse<min_error:
            min_error=rmse
            lstm_best_model=best_model
          #mape=mean_absolute_percentage_error(test_y,predictions)
          r2s=r2_score(test_y,predictions)
          results_lstm.append([state,n_f,t_s,n_layers,n_hidden_nodes,mae,rmse,r2s])
          print(state,'n_f',n_f,'t_s',t_s,'n_layers',n_layers,n_hidden_nodes,'Error',mae,rmse,r2s)
  lstm_models.append(lstm_best_model) 

(384, 5, 1)
(384, 5, 1) (384, 1)
1
Epoch: 50/100  Loss: 0.146149 Test loss: 0.3137311
Karnataka n_f 1 t_s 5 n_layers 1 1 Error 0.5067914 0.556090715726424 -4.549494111340635
1
Epoch: 50/100  Loss: 0.105253 Test loss: 0.1377941
Karnataka n_f 1 t_s 5 n_layers 1 5 Error 0.17510131 0.20644651464944141 0.23514839320520986
1
Epoch: 0/100  Loss: 0.169845 Test loss: 0.392808

KeyboardInterrupt: ignored